In [1]:
# Non cross term differential forms in matrix representation

# define our root of unity:
var('w')
W = exp(I*2*pi/3)
k = CyclotomicField(3)
Z = k.gen()


# variables to help with walking the parse tree:
var('sym1 sym2')

# Now on to some helper functions:
# Noting that q = 3 requires more help than q = 2
# define our w_modulus function:
def w_mod(r):
    r = expand(r)
    try:
        if len(r.operands()) == 0:
            return r
    except:
        return r
    if r.operator() == (sym1 + sym2).operator():
        result = 0
        for term in r.operands():
            result += w_mod(term)
        return result
    if r.operator() == (sym1*sym2).operator():
        result = 1
        for term in r.operands():
            result *= w_mod(term)
        return result
    if r.operator() == (sym1^sym2).operator():
        head = r.operands()[0]
        tail = r.operands()[1]
        if head != w:
            return r
        try:
            tail = Mod(tail, 3)
            return head^tail
        except:
            return r

# wrapper for our w_mod function, so we can apply it to matrices:
def matrix_w_mod(A):
    return A.apply_map(w_mod)


# Implement 1 + w + w^2 = 0
# Needs verifying!
def expand_w(r):
    r = expand(r)
    if r.operator() != (sym1 + sym2).operator():  # Already compact, so return it unchanged
        return r
    non_w2_terms = r.substitute({w^2: 0})
    w2_terms = r - non_w2_terms
    w2_terms = expand(w2_terms.substitute({w: 1}))
    r2 = expand(non_w2_terms - w2_terms*(1 + w))
    if r2.operator() != (sym1 + sym2).operator(): # Compact, so return the new result
        return r2
    if len(r2.operands()) < len(r.operands()):  # Smaller, so let's keep this change
        non_w_terms = r2.substitute({w: 0})
        w_terms = r2 - non_w_terms
        w_terms = expand(w_terms.substitute({w: 1}))
        if w_terms.operator() != (sym1 + sym2).operator():
            r3 = expand(non_w_terms - w_terms*(1 + w^2))
            if r3.operator() != (sym1 + sym2).operator():
                return r3
            if len(r3.operands()) < len(r2.operands()):
                return r3
        result = r2
        for t in w_terms.operands():
            r3 = expand(result - t*(1 + w + w^2))
            if r3.operator() != (sym1 + sym2).operator():
                return r3
            if len(r3.operands()) < len(result.operands()):
                result = r3
        if len(result.operands()) < len(r2.operands()):
            return result
        return r2
    if w2_terms == 0:
        non_w_terms = r.substitute({w: 0})
        w_terms = r - non_w_terms
        w_terms = expand(w_terms.substitute({w: 1}))
        r2 = expand(non_w_terms - w_terms*(1 + w^2))
        if r2.operator() != (sym1 + sym2).operator():
            return r2
        if len(r2.operands()) < len(r.operands()):
            return r2
        return r
    return r
    
def layer2_expand_w(r):
    r = w_mod(r)
    r = expand_w(r)
    if r.operator() != (sym1 + sym2).operator():  # Already compact, so return it unchanged
        return r
    test_r = w_mod(w^2*(expand_w(w_mod(w*r))))
    if test_r.operator() != (sym1 + sym2).operator():
        return test_r
    if len(test_r.operands()) < len(r.operands()):
        return test_r
    return r


def matrix_expand_w(A):
    return A.apply_map(expand_w)


In [4]:
# Define our 3x3 matrices:
N1 = matrix([[0,1,0],[0,0,1],[1,0,0]])
N2 = matrix([[0,1,0],[0,0,w^2],[w,0,0]])
N3 = matrix([[1,0,0],[0,w^2,0],[0,0,w]])
I3 = matrix([[1,0,0],[0,1,0],[0,0,1]])

# Display them:
show('dim 3 matrices:')
show('N1:', N1)
show('N2:', N2)
show('N3:', N3)

# Check their commutator properties match those required for differential forms:
show('Their commutator properties:')
show('N3 N1 - w N1 N3:', matrix_w_mod(N3*N1 - w * N1 * N3))
show('N2 N1 - w N1 N2:', matrix_w_mod(N2*N1 - w * N1 * N2))
show('N3 N2 - w N2 N3:', matrix_w_mod(N3*N2 - w * N2 * N3))

# Check they are matrix cube root:
var('p1 p2 p3')
r = p1*N1 + p2*N2 + p3*N3
show('Check they are a matrix cube root:')
show('r^3:', matrix_expand_w(matrix_w_mod(r^3)))

show('-------------------------------------')
# Define our 9x9 matrices:
I9 = I3.tensor_product(I3)
NN0 = N1.tensor_product(I3)
NN1 = N3.tensor_product(N1)
NN2 = N3.tensor_product(N2)
NN3 = N3.tensor_product(N3)

show('dim 4 matrices:')
show('NN0:', NN0)
show('NN1:', NN1)
show('NN2:', NN2)
show('NN3:', NN3)

# Check their commutator properties:
show('Their commutator properties:')
show('NN1 NN0 - w NN0 NN1:', matrix_w_mod(NN1*NN0 - w * NN0 * NN1))
show('NN2 NN0 - w NN0 NN2:', matrix_w_mod(NN2*NN0 - w * NN0 * NN2))
show('NN3 NN0 - w NN0 NN3:', matrix_w_mod(NN3*NN0 - w * NN0 * NN3))
show('  ')
show('NN3 NN1 - w NN1 NN3:', matrix_w_mod(NN3*NN1 - w * NN1 * NN3))
show('NN2 NN1 - w NN1 NN2:', matrix_w_mod(NN2*NN1 - w * NN1 * NN2))
show('NN3 NN2 - w NN2 NN3:', matrix_w_mod(NN3*NN2 - w * NN2 * NN3))

show('----------------------')
var('p0')
r = p0*NN0 + p1*NN1 + p2*NN2 + p3*NN3
show('Check they are a matrix cube root:')
show('r^3:', matrix_expand_w(matrix_w_mod(r^3)))

'dim 3 matrices:'

'N1:' [0 1 0]
[0 0 1]
[1 0 0]

'N2:' [  0   1   0]
[  0   0 w^2]
[  w   0   0]

'N3:' [  1   0   0]
[  0 w^2   0]
[  0   0   w]

'Their commutator properties:'

...

'NN2 NN1 - w NN1 NN2:' [0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]

'NN3 NN2 - w NN2 NN3:' [0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]

'----------------------'

'Check they are a matrix cube root:'

'r^3:' [p0^3 + p1^3 + p2^3 + p3^3                         0                         0                         0                         0                         0                         0                         0                         0]
[                        0 p0^3 + p1^3 + p2^3 + p3^3                         0                         0                         0                         0                         0                         0                         0]
[                        0                         0 p0^3 + p1^3 + p2^3 + p3^3                         0                         0                         0                         0                         0                         0]
[                        0                         0                         0 p0^3 + p1^3 + p2^3 + p3^3                         0                         0                         0                         0                         0]
[                        0                         0                         0                         0 p0^3 + p1^3 + p2^3 + p3^3                         0                         0                         0                         0]
[                        0                         0                         0                         0                         0 p0^3 + p1^3 + p2^3 + p3^3                         0                         0                         0]
[                        0                         0                         0                         0                         0                         0 p0^3 + p1^3 + p2^3 + p3^3                         0                         0]
[                        0                         0                         0                         0                         0                         0                         0 p0^3 + p1^3 + p2^3 + p3^3                         0]
[                        0                         0                         0                         0                         0                         0                         0                         0 p0^3 + p1^3 + p2^3 + p3^3]

In [5]:
# Now, let's generate the basis matrices:
import itertools


# Display the k-forms dimensions table given d and q, and return the basis_dict:
def dim_table(d, q):
    basis_indices = []
    if d == 3 and q == 2:
        basis_indices = [1,2,3]
    if d == 4 and q == 2:
        basis_indices = [0,1,2,3]
    if d == 3 and q == 3:
        basis_indices = [1,2,3]
    if d == 4 and q == 3:
        # basis_indices = [0,1,2,3,4,5,6]
        basis_indices = [0,1,2,3]
    basis_dict = {}
    k = 0
    while True:
        basis_dict[k] = []
        basis_count = 0
        for elt in itertools.combinations_with_replacement(basis_indices, k):
            match = False
            for b in basis_indices:
                if elt.count(b) >= q:
                    match = True
                    break
            if match:
                continue
            # print(elt)
            basis_dict[k] += [elt]
            basis_count += 1
        k += 1
        if basis_count == 0:
            break
    rows = [['k', 'dim', 'basis']]
    for i in range(k - 1):
        basis_result = basis_dict[i]
        str_basis_result = " ".join("".join(str(x) for x in b) for b in basis_result)
        rows += [[i, len(basis_result), str_basis_result]]
    show(table(rows, header_row=True))
    return basis_dict


# generate the d = 4, q = 3 basis dictionary:
show('The dim = 4, q = 3 k-form dimension table:')
basis4_dict = dim_table(4, 3)


# Extract out all the basis labels:
def generate_basis_labels(basis_dict):
    basis_labels = {}
    for i in basis_dict:
        basis_labels[i] = ['D' + "".join(str(x) for x in b) for b in basis_dict[i]]
    return basis_labels

basis4_labels = generate_basis_labels(basis4_dict)

# Given the matrix indices, return the resulting basis matrix, here for d = 3:
def D3(indices):
    # matrices = [T1, T2, T3]
    matrices = [N1, N2, N3]
    r = I3
    for i in indices:
        r = r*matrices[i - 1]
    # show(r)
    return matrix_w_mod(expand(r))

def D4(indices):
    # matrices = [S0, S1, S2, S3, S4, S5, S6]
    matrices = [NN0, NN1, NN2, NN3]
    r = I9
    for i in indices:
        r = r*matrices[i]
    # show(r)
    return matrix_w_mod(expand(r))

# Quick test of basis4_labels:
show('k = 2 basis4 labels:')
show(basis4_labels[2])

# Learn all the d = 4, q = 3, basis matrices:
# NB; makes generate_basis_labels() redundant.
def generate_basis4_matrices(basis_dict):
    basis_labels = {}
    basis_matrices = {}
    for i in basis_dict:
        basis_labels[i] = []
        for b in basis_dict[i]:
            label = 'D' + "".join(str(x) for x in b)
            matrix = D4(list(b))
            # show(list(b), matrix)
            basis_labels[i] += [label]
            basis_matrices[label] = matrix
    return basis_labels, basis_matrices

basis4_labels, basis4_matrices = generate_basis4_matrices(basis4_dict)


# Explore the basis overlap property:
# Test if two matrices are equal up to some w coefficient:
def find_k(A,B):
    for k in range(3):
        if matrix_w_mod(A - w^k*B) == 0:
            return k
        if matrix_w_mod(A + w^k*B) == 0:
            return -k
    return None

# Find the commutator structure for a pair of matrices:
# Ie, A*B = (w)^k B*A
def find_commutator(A,B):
    for k in range(3):
        if matrix_w_mod(A*B - w^k*B*A) == 0:
            return k
    return None

# Get a unique basis given a k:
def get_unique_basis4(k):
    if k not in basis4_labels:
        return
    result = []
    for label in basis4_labels[k]:
        # show(label)
        matrix = basis4_matrices[label]
        match = False
        for A_label in result:
            A = basis4_matrices[A_label]
            # show('A:', A)
            # show(find_k(A, matrix))
            if find_k(A, matrix) is not None:
                match = True
                break
        if not match:
            result += [label]
    return result

# Learn unique basis labels all at once:
def generate_unique_basis4_labels():
    unique_basis_labels = {}
    for k in range(16):
        unique_basis_labels[k] = get_unique_basis4(k)
    return unique_basis_labels

unique_basis4_labels = generate_unique_basis4_labels()

# Quick test:
show('k = 2 unique basis4 labels:')
show(unique_basis4_labels[2])

# Display the unique dimension table:
def unique_dim_table(basis_dict):
    rows = [['k', 'dim', 'unique dim']]
    for k in basis_dict:
        # rows += [[k, len(basis_dict[k]), len(get_unique_basis4(k))]]
        rows += [[k, len(basis_dict[k]), len(unique_basis4_labels[k])]]
    show(table(rows, header_row=True))
    return latex(table(rows, header_row=True))
    
show('----------------------------------')
show('The unique k-form dimension table:')
unique_dim_table(basis4_dict)

'The dim = 4, q = 3 k-form dimension table:'

k,dim,basis
,,
,,0 1 2 3
,,00 01 02 03 11 12 13 22 23 33
,,001 002 003 011 012 013 022 023 033 112 113 122 123 133 223 233
,,0011 0012 0013 0022 0023 0033 0112 0113 0122 0123 0133 0223 0233 1122 1123 1133 1223 1233 2233
,,00112 00113 00122 00123 00133 00223 00233 01122 01123 01133 01223 01233 02233 11223 11233 12233
,,001122 001123 001133 001223 001233 002233 011223 011233 012233 112233
,,0011223 0011233 0012233 0112233
,,00112233


'k = 2 basis4 labels:'

['D00', 'D01', 'D02', 'D03', 'D11', 'D12', 'D13', 'D22', 'D23', 'D33']

'k = 2 unique basis4 labels:'

['D00', 'D01', 'D02', 'D03', 'D11', 'D12', 'D13', 'D22', 'D23', 'D33']

'----------------------------------'

'The unique k-form dimension table:'

k,dim,unique dim
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


\begin{tabular}{lll}
k & dim & unique dim \\ \hline
$0$ & $1$ & $1$ \\
$1$ & $4$ & $4$ \\
$2$ & $10$ & $10$ \\
$3$ & $16$ & $16$ \\
$4$ & $19$ & $19$ \\
$5$ & $16$ & $16$ \\
$6$ & $10$ & $10$ \\
$7$ & $4$ & $4$ \\
$8$ & $1$ & $1$ \\
$9$ & $0$ & $0$ \\
\end{tabular}

In [7]:
def commutation_table(labels):
    show(table([[find_commutator(basis4_matrices[y],basis4_matrices[x]) for y in labels] for x in labels], header_column=[' '] + labels, header_row=labels))
    return latex(table([[find_commutator(basis4_matrices[y],basis4_matrices[x]) for y in labels] for x in labels], header_column=[' '] + labels, header_row=labels))

show('commutation table for 1-form matrices:')
commutation_table(unique_basis4_labels[1])

'commutation table for 1-form matrices:'

,D0,D1,D2,D3
D0,,,,
D1,,,,
D2,,,,
D3,,,,


\begin{tabular}{l|llll}
  & D0 & D1 & D2 & D3 \\ \hline
D0 & $0$ & $1$ & $1$ & $1$ \\
D1 & $2$ & $0$ & $1$ & $1$ \\
D2 & $2$ & $2$ & $0$ & $1$ \\
D3 & $2$ & $2$ & $2$ & $0$ \\
\end{tabular}

In [8]:
# Let's write the custum_latex(r) code:
# Converts an expression into correctly sorted and formatted latex

# Define our variables:
var('d0 d1 d2 d3 d4 d5 d6')
var('A0 A1 A2 A3 A4 A5 A6')

# The order of the terms in this list, is the order they will be sorted to in expressions:
custom_latex_sort_order = [-1, w, w^2]
custom_latex_sort_order += [d0, d1, d2, d3, d4, d5, d6]
custom_latex_sort_order += [A0, A1, A2, A3, A4, A5, A6]

# Maps indecies to their associated latex:
custom_latex_string = ['-', 'w', 'w^2']
custom_latex_string += [r'\partial_0', r'\partial_1', r'\partial_2', r'\partial_3', r'\partial_4', r'\partial_5', r'\partial_6']
custom_latex_string += ['A_0', 'A_1', 'A_2', 'A_3', 'A_4', 'A_5', 'A_6']

# The function itself:
def custom_latex(r):
    r = expand(r)
    latex_result = ''
    if r.operator() == (sym1 * sym2).operator():
        # show('r.operands:', r.operands())
        match = False
        for x in r.operands():
            if x < 0:
                match = True
        if match:
            # show('Match!')
            latex_result += ' - ' + custom_latex(-r)
        else:
            idx_list = []
            power_term_tails = {}
            number = 1
            for t in r.operands():
                # show('t:', t)
                if t.operator() == (sym1^sym2).operator():
                    head, tail = t.operands()
                    idx = custom_latex_sort_order.index(head)
                    idx_list.append(idx)
                    power_term_tails[idx] = tail
                else:
                    if t not in custom_latex_sort_order:
                        number = t
                    else:
                        idx = custom_latex_sort_order.index(t)
                        idx_list.append(idx)
            idx_list.sort()
            result_list = []
            if number != 1:
                result_list.append(latex(number))
            for i in idx_list:
                if i in power_term_tails:
                    result_list.append(custom_latex_string[i] + '^{' + latex(power_term_tails[i]) + '}')
                else:
                    result_list.append(custom_latex_string[i])
            latex_result += ' '.join(x for x in result_list)
    elif r.operator() == (sym1 + sym2).operator():
        latex_result += ' + '.join(custom_latex(term) for term in r.operands())
    elif r.operator() == (sym1^sym2).operator():
        head, tail = r.operands()
        head_idx = custom_latex_sort_order.index(head)
        latex_result += '%s^{%s}' % (custom_latex_string[head_idx], latex(tail))
    else:
        idx = custom_latex_sort_order.index(r)
        latex_result += custom_latex_string[idx]
    return latex_result
    # return latex_result.replace(' + - ', ' - ')
            
# Quick test:    
# custom_latex(2*w)
# custom_latex(- 2* w*d5^7)
show('Quick test of custom latex function:')
print(custom_latex(A5*d3*w^2 + A2*d4 - 3.14* d5^7 + w + A3 + w^2))

'Quick test of custom latex function:'

 - 3.14000000000000 \partial_5^{ 7 } + w^{ 2 } \partial_3 A_5 + \partial_4 A_2 + w^{2} + A_3 + w


In [10]:
# Now extract out basis terms:
def get_basis4(k, A):
    if k not in unique_basis4_labels:
        return
    labels = unique_basis4_labels[k]
    latex_result = ''
    results = {}
    for label in labels:
        matrix = basis4_matrices[label]
        term = (expand(A*matrix^5)).trace()/9
        term = w_mod(term)
        # term = expand_w(term)
        term = layer2_expand_w(term)
        term = expand(term)
        # term = expand_w(term)
        term = layer2_expand_w(term)
        if term != 0:
            show(label, '::', term)
            results[label] = term
            latex_result += '& ' + latex(label + ': ') + custom_latex(term) + '\\\\\n'
    print(latex_result)
    return results

# Now a quick test:
var('A0 A1 A2 A3')
A = A0*NN0 + A1*NN1 + A2*NN2 + A3*NN3
show('Quick test of get_basis4(k, A), for A a 1-form:')
get_basis4(1, A)

show('Quick test of dA as a 2-form:')
d = d0*NN0 + d1*NN1 + d2*NN2 + d3*NN3
get_basis4(2, d*A)

'Quick test of get_basis4(k, A), for A a 1-form:'

'D0' '::' A0

'D1' '::' A1

'D2' '::' A2

'D3' '::' A3

...

'D22' '::' A2*d2

'D23' '::' A2*d3*w + A3*d2

'D33' '::' A3*d3

& \text{\texttt{D00:{ }}} \partial_0 A_0 \\
 & \text{\texttt{D01:{ }}} w \partial_1 A_0 + \partial_0 A_1 \\
 & \text{\texttt{D02:{ }}} w \partial_2 A_0 + \partial_0 A_2 \\
 & \text{\texttt{D03:{ }}} w \partial_3 A_0 + \partial_0 A_3 \\
 & \text{\texttt{D11:{ }}} \partial_1 A_1 \\
 & \text{\texttt{D12:{ }}} w \partial_2 A_1 + \partial_1 A_2 \\
 & \text{\texttt{D13:{ }}} w \partial_3 A_1 + \partial_1 A_3 \\
 & \text{\texttt{D22:{ }}} \partial_2 A_2 \\
 & \text{\texttt{D23:{ }}} w \partial_3 A_2 + \partial_2 A_3 \\
 & \text{\texttt{D33:{ }}} \partial_3 A_3 \\



{'D00': A0*d0,
 'D01': A0*d1*w + A1*d0,
 'D02': A0*d2*w + A2*d0,
 'D03': A0*d3*w + A3*d0,
 'D11': A1*d1,
 'D12': A1*d2*w + A2*d1,
 'D13': A1*d3*w + A3*d1,
 'D22': A2*d2,
 'D23': A2*d3*w + A3*d2,
 'D33': A3*d3}